In [ ]:
#Application of SVM using tensorflow, seperated aout this procedure to reduce clutter
#For brevity, we use linear seperator kernels, obvious from the 3d plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops

from pandas import Series, DataFrame

ops.reset_default_graph()

from pylab import *
import matplotlib.pyplot as plt

%pylab inline

In [ ]:
data_file = pd.read_csv("train.csv")
data_file = data_file.drop(['Ticket','Cabin','PassengerId','Name'], axis=1)
data_file = data_file.dropna()

def to_numeric_str(var):
    if var=='male':
        return 0
    else:
        return 1

    
data_file['Sex_num'] = 0
data_file[['Sex_num']] = data_file[['Sex']].apply(lambda row: to_numeric_str(row['Sex']),axis=1)
data_file = data_file.drop(['Sex'],axis=1)
data_file[0:5]

In [ ]:
data = data_file.as_matrix()
x_data = np.array([[x[1],x[2],x[3],x[4],x[5],x[7]] for x in data])
#Converting into classes 1 and -1
y_data = np.array([1 if x[0]==1 else -1 for x in data])

#Splitting data into train/test 90 and 10 percent
train_idx = np.random.choice(len(x_data), int(round(len(x_data)*0.9)), replace=False)
test_idx = np.array(list(set(range(len(x_data))) - set(train_idx)))
x_train = x_data[train_idx]
y_train = y_data[train_idx]

x_test = x_data[test_idx]
y_test = y_data[test_idx]


In [ ]:
batch_size = 10
learning_rate = 0.001

graph_ = tf.Graph()
with graph_.as_default():
    #Creation of tensorflow graph
    with tf.device('/cpu:0'):
        x_feed = tf.placeholder(shape = [None, 6], dtype = tf.float32)
        y_feed = tf.placeholder(shape = [None, 1], dtype = tf.float32)

        #Declaration of seperator, for margin maximization
        A = tf.Variable(tf.random_normal(shape=[6,1]))
        b = tf.Variable(tf.random_normal(shape=[1,1]))

        #Eqation of line, y = Ax-b
        out = tf.subtract(tf.matmul(x_feed,A),b)

        #Maximize the norm(W)^2, here it is just W = A
        l2_norm = tf.reduce_sum(tf.square(A))

        #Additionaly, for regularization, we add some more paramters
        #regularization_term = max(0, 1-pred*actual)
        alpha = tf.constant([0.001])
        regularization_term = tf.reduce_mean(tf.maximum(0., tf.subtract(1., tf.multiply(out,y_feed))))

        #Loss function
        loss = tf.add(regularization_term, tf.multiply(alpha, l2_norm))
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)

        #Defining varibales of interest
        train_model = optimizer.minimize(loss)

        #We are interested in sign, depicting two classes
        prediction = tf.sign(out)
        nr_correct = tf.cast(tf.equal(prediction, y_feed), tf.float32)
        accuracy = tf.reduce_mean(nr_correct)


In [ ]:

with tf.Session(graph=graph_,config=tf.ConfigProto(log_device_placement=True)) as sess:
    tf.initialize_all_variables().run()
    print('Initialized')
    for i in range(100000):
        random_batch_choice = np.random.choice(len(x_train), size = batch_size)
        random_x = x_train[random_batch_choice]
        random_y = np.transpose([y_train[random_batch_choice]])
        sess.run(train_model, feed_dict={x_feed:random_x, y_feed:random_y})
        test_accuracy = sess.run(accuracy, feed_dict={x_feed:x_test, y_feed:np.transpose([y_test])})
        if(i%5000==0):
            print("Test accuracy at step ", i, " is ",test_accuracy)
    #Get the answer for real data
    #ans = sess.run(prediction, feed_dict={x_feed:x_data_ans})
    #ans[0:5]

In [ ]:
#Next to try: Non-linear SVM kernel
#Next to next: Decision trees